In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/judgement/sample_submission.csv
/kaggle/input/judgement/train.csv
/kaggle/input/judgement/test.csv


In [2]:
train = pd.read_csv('/kaggle/input/judgement/train.csv')
test = pd.read_csv('/kaggle/input/judgement/test.csv')
display(train,test)

,ID,first_party,second_party,facts,first_party_winner
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,0
2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,1
3,TRAIN_0003,Linkletter,Walker,Victor Linkletter was convicted in state court...,0
4,TRAIN_0004,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",1
...,...,...,...,...,...
2473,TRAIN_2473,"HollyFrontier Cheyenne Refining, LLC, et al.","Renewable Fuels Association, et al.",Congress amended the Clean Air Act through the...,1
2474,TRAIN_2474,"Grupo Mexicano de Desarrollo, S. A.","Alliance Bond Fund, Inc.","Alliance Bond Fund, Inc., an investment fund, ...",1
2475,TRAIN_2475,Peguero,United States,"In 1992, the District Court sentenced Manuel D...",0
2476,TRAIN_2476,Immigration and Naturalization Service,St. Cyr,"On March 8, 1996, Enrico St. Cyr, a lawful per...",0


,ID,first_party,second_party,facts
0,TEST_0000,Salerno,United States,The 1984 Bail Reform Act allowed the federal c...
1,TEST_0001,Milberg Weiss Bershad Hynes and Lerach,"Lexecon, Inc.",Lexecon Inc. was a defendant in a class action...
2,TEST_0002,No. 07-582\t Title: \t Federal Communications ...,"Fox Television Stations, Inc., et al.","In 2002 and 2003, Fox Television Stations broa..."
3,TEST_0003,Harold Kaufman,United States,During his trial for armed robbery of a federa...
4,TEST_0004,Berger,Hanlon,"In 1993, a magistrate judge issued a warrant a..."
...,...,...,...,...
1235,TEST_1235,"Haitian Centers Council, Inc., et al.","Chris Sale, Acting Commissioner, Immigration A...",According to Executive Order No. 12807 signed ...
1236,TEST_1236,Whitman,"American Trucking Associations, Inc.",Section 109(a) of the Clean Air Act (CAA) requ...
1237,TEST_1237,Linda A. Matteo and John J. Madigan,William G. Barr,Linda Matteo and John Madigan created a plan f...
1238,TEST_1238,Washington State Apple Advertising Commission,Hunt,"In 1972, the North Carolina Board of Agricultu..."


In [7]:
#* 맞출 값 : first_party 승리 여부
#* first_party와 fact의 유사도 구해서 칼럼 추가 -> 머신러닝

In [4]:
all_data = pd.concat([train, test])
all_data

,ID,first_party,second_party,facts,first_party_winner
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1.0
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,0.0
2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,1.0
3,TRAIN_0003,Linkletter,Walker,Victor Linkletter was convicted in state court...,0.0
4,TRAIN_0004,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",1.0
...,...,...,...,...,...
1235,TEST_1235,"Haitian Centers Council, Inc., et al.","Chris Sale, Acting Commissioner, Immigration A...",According to Executive Order No. 12807 signed ...,NaN
1236,TEST_1236,Whitman,"American Trucking Associations, Inc.",Section 109(a) of the Clean Air Act (CAA) requ...,NaN
1237,TEST_1237,Linda A. Matteo and John J. Madigan,William G. Barr,Linda Matteo and John Madigan created a plan f...,NaN
1238,TEST_1238,Washington State Apple Advertising Commission,Hunt,"In 1972, the North Carolina Board of Agricultu...",NaN


In [5]:
#* first_party 
def normalized_word_share(row): #* 단어가 얼마나 겹치는가? 중복되는가?
    w1 = set(map(lambda word: word.lower().strip(), row['first_party'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['facts'].split(" ")))    
    return 1.0 * len(w1 & w2)/(len(w1) + len(w2))

In [6]:
all_data['first_party_share'] = all_data.apply(normalized_word_share, axis = 1) #* 칼럼 방향 접근
all_data

,ID,first_party,second_party,facts,first_party_winner,first_party_share
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1.0,0.025641
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,0.0,0.000000
2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,1.0,0.028846
3,TRAIN_0003,Linkletter,Walker,Victor Linkletter was convicted in state court...,0.0,0.024390
4,TRAIN_0004,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",1.0,0.024590
...,...,...,...,...,...,...
1235,TEST_1235,"Haitian Centers Council, Inc., et al.","Chris Sale, Acting Commissioner, Immigration A...",According to Executive Order No. 12807 signed ...,NaN,0.037037
1236,TEST_1236,Whitman,"American Trucking Associations, Inc.",Section 109(a) of the Clean Air Act (CAA) requ...,NaN,0.000000
1237,TEST_1237,Linda A. Matteo and John J. Madigan,William G. Barr,Linda Matteo and John Madigan created a plan f...,NaN,0.038462
1238,TEST_1238,Washington State Apple Advertising Commission,Hunt,"In 1972, the North Carolina Board of Agricultu...",NaN,0.031250


In [8]:
def normalized_word_share2(row): #* 단어가 얼마나 겹치는가? 중복되는가?
    w1 = set(map(lambda word: word.lower().strip(), row['second_party'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['facts'].split(" ")))    
    return 1.0 * len(w1 & w2)/(len(w1) + len(w2)) 

In [9]:
all_data['second_party_share'] = all_data.apply(normalized_word_share2, axis = 1) #* 칼럼 방향 접근
all_data

,ID,first_party,second_party,facts,first_party_winner,first_party_share,second_party_share
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1.0,0.025641,0.017241
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,0.0,0.000000,0.016667
2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,1.0,0.028846,0.000000
3,TRAIN_0003,Linkletter,Walker,Victor Linkletter was convicted in state court...,0.0,0.024390,0.000000
4,TRAIN_0004,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",1.0,0.024590,0.008333
...,...,...,...,...,...,...,...
1235,TEST_1235,"Haitian Centers Council, Inc., et al.","Chris Sale, Acting Commissioner, Immigration A...",According to Executive Order No. 12807 signed ...,NaN,0.037037,0.017857
1236,TEST_1236,Whitman,"American Trucking Associations, Inc.",Section 109(a) of the Clean Air Act (CAA) requ...,NaN,0.000000,0.000000
1237,TEST_1237,Linda A. Matteo and John J. Madigan,William G. Barr,Linda Matteo and John Madigan created a plan f...,NaN,0.038462,0.023810
1238,TEST_1238,Washington State Apple Advertising Commission,Hunt,"In 1972, the North Carolina Board of Agricultu...",NaN,0.031250,0.000000


In [13]:
all_data2 = all_data[['first_party_share','second_party_share']]

In [14]:
train2 = all_data2[:len(train)]
test2 = all_data2[len(train):]
display(train2, test2)

,first_party_share,second_party_share
0,0.025641,0.017241
1,0.000000,0.016667
2,0.028846,0.000000
3,0.024390,0.000000
4,0.024590,0.008333
...,...,...
2473,0.000000,0.020202
2474,0.042017,0.025641
2475,0.010204,0.000000
2476,0.018182,0.018519


,first_party_share,second_party_share
0,0.022727,0.000000
1,0.059406,0.010309
2,0.017094,0.017544
3,0.046154,0.000000
4,0.000000,0.000000
...,...,...
1235,0.037037,0.017857
1236,0.000000,0.000000
1237,0.038462,0.023810
1238,0.031250,0.000000


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state = 72)
result = 0
for train_index, valid_index in skf.split(train2, train['first_party_winner']):
    X_train, X_valid = train2.iloc[train_index], train2.iloc[valid_index]
    y_train, y_valid = train['first_party_winner'][train_index], train['first_party_winner'][valid_index]
    cbc = CatBoostClassifier(verbose = 100)
    cbc.fit(X_train, y_train, eval_set = (X_valid, y_valid), early_stopping_rounds = 100)
    result += cbc.predict_proba(test2) / 10

Learning rate set to 0.038631
0:	learn: 0.6885946	test: 0.6887910	best: 0.6887910 (0)	total: 56.5ms	remaining: 56.4s
100:	learn: 0.6166903	test: 0.6421463	best: 0.6409571 (53)	total: 207ms	remaining: 1.85s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6409570884
bestIteration = 53

Shrink model to first 54 iterations.
Learning rate set to 0.038631
0:	learn: 0.6887155	test: 0.6888270	best: 0.6888270 (0)	total: 1.31ms	remaining: 1.31s
100:	learn: 0.6187080	test: 0.6348811	best: 0.6329813 (66)	total: 153ms	remaining: 1.36s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6329813465
bestIteration = 66

Shrink model to first 67 iterations.
Learning rate set to 0.038631
0:	learn: 0.6885091	test: 0.6889907	best: 0.6889907 (0)	total: 1.82ms	remaining: 1.82s
100:	learn: 0.6149510	test: 0.6427963	best: 0.6389047 (44)	total: 142ms	remaining: 1.27s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6389047312
bestIteration = 44

Shrink m

In [19]:
sub = pd.read_csv('/kaggle/input/judgement/sample_submission.csv')
sub

,ID,first_party_winner
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0
...,...,...
1235,TEST_1235,0
1236,TEST_1236,0
1237,TEST_1237,0
1238,TEST_1238,0


In [21]:
sub['first_party_winner'] = result[:, 1] > 0.6 
sub['first_party_winner'].value_counts()
sub

,ID,first_party_winner
0,TEST_0000,False
1,TEST_0001,True
2,TEST_0002,True
3,TEST_0003,True
4,TEST_0004,True
...,...,...
1235,TEST_1235,True
1236,TEST_1236,True
1237,TEST_1237,True
1238,TEST_1238,False


In [22]:
sub.to_csv('sub4.csv', index=False)